In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [15]:
'''A pivot table is a similar operation that is commonly seen in spreadsheets and
other programs that operate on tabular data. The pivot table takes simple column-wise data 
as input, and groups the entries into a two-dimensional table that provides
a multidimensional summarization of the data. '''

# Pivot Tables by Hand

titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [22]:
titanic.groupby(['sex', 'class'])[['survived']].aggregate('mean').unstack()

survived                    
class      First    Second     Third
sex                                 
female  0.968085  0.921053  0.500000
male    0.368852  0.157407  0.135447

In [7]:
# counting the number of people who boarded from different towns

titanic.embark_town.value_counts()

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64

In [15]:
# getting the number of people on board based on gender

titanic.sex.value_counts()

male      577
female    314
Name: sex, dtype: int64

In [135]:
# Pivot Table Syntax
# pivot_table is used to summarize and aggregate data in a DataFrame

titanic.pivot_table('survived', index = 'sex', columns = 'class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [64]:
# Multilevel pivot tables
#Just as in the GroupBy, the grouping in pivot tables can be specified with multiple levels, and via a number of options

# We’ll bin the age using the pd.cut function:
age = pd.cut(titanic['age'],[0, 18, 50, 80])
titanic.pivot_table('survived', ['sex', age], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 50]  0.967213  0.912281  0.413793
       (50, 80]  1.000000  0.666667  1.000000
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 50]  0.442857  0.069444  0.139896
       (50, 80]  0.192308  0.083333  0.000000

In [66]:
# We can apply this same strategy when working with the columns as well

fare = pd.cut(titanic['fare'],[0, 200, 400])
titanic.pivot_table('survived', ['sex', age], ['class', fare])

class               First               Second     Third
fare             (0, 200] (200, 400]  (0, 200]  (0, 200]
sex    age                                              
female (0, 18]   0.875000        1.0  1.000000  0.511628
       (18, 50]  0.961538        1.0  0.912281  0.413793
       (50, 80]  1.000000        NaN  0.666667  1.000000
male   (0, 18]   0.800000        NaN  0.600000  0.215686
       (18, 50]  0.467742        0.0  0.069444  0.137566
       (50, 80]  0.200000        0.0  0.083333  0.000000

In [71]:
# let’s add info on the fare paid using pd.qcut to automatically compute quantiles:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 50]              NaN  0.916667  0.428571          0.967213   
       (50, 80]              NaN  0.000000  1.000000          1.000000   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 50]              0.0  0.095238  0.131737          0.469697   
       (50, 80]              NaN  0.111111  0.000000          0.192308   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 50]  0.909091  0.391304  
       (50, 80]  1.000000       NaN  
male   (0, 18]   0.818182  0.178571  
       (18, 50]  0.033333  0.192308  
       (50, 80]  0.000000       NaN

In [79]:
# using the aggfunc attribute with multiple arguments

titanic.pivot_table(index = 'sex', columns = 'class', aggfunc = {'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

In [82]:
# the aggfunc taking one argument

titanic.pivot_table('fare', index = 'sex', columns = 'class', aggfunc = sum)

class,First,Second,Third
sex,,,
female,9975.8250,1669.7292,2321.1086
male,8201.5875,2132.1125,4393.5865


In [86]:
# marging argument 

titanic.pivot_table('survived', index = 'sex', columns = 'class', margins = True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


In [94]:
#sample data 
# appplying pivot function

births = pd.read_csv('births.csv')

print(births.shape)
births.head()

(15547, 5)


,year,month,day,gender,births
0,1969,1,1.0,F,4046
1,1969,1,1.0,M,4440
2,1969,1,2.0,F,4454
3,1969,1,2.0,M,4548
4,1969,1,3.0,F,4548


In [104]:
# assigning decade column on the data set 

births['decade'] = 10 * (births['year'] // 10)
births.pivot_table('births', index = 'decade', columns = 'gender', aggfunc = sum)

gender,F,M
decade,,
1960,1753634,1846572
1970,16263075,17121550
1980,18310351,19243452
1990,19479454,20420553
2000,18229309,19106428


In [5]:
# Pivot allows you to transform or reshape data
# acts in a similar manner to multi indexing

stocks = pd.read_csv('http://bit.ly/smallstocks')
stocks

,Date,Close,Volume,Symbol
0,2016-10-03,31.50,14070500,CSCO
1,2016-10-03,112.52,21701800,AAPL
2,2016-10-03,57.42,19189500,MSFT
3,2016-10-04,113.00,29736800,AAPL
4,2016-10-04,57.24,20085900,MSFT
5,2016-10-04,31.35,18460400,CSCO
6,2016-10-05,57.64,16726400,MSFT
7,2016-10-05,31.59,11808600,CSCO
8,2016-10-05,113.05,21453100,AAPL


In [121]:
stocks.pivot(index = 'Date', columns = 'Symbol')

Close                  Volume                    
Symbol        AAPL   CSCO   MSFT      AAPL      CSCO      MSFT
Date                                                          
2016-10-03  112.52  31.50  57.42  21701800  14070500  19189500
2016-10-04  113.00  31.35  57.24  29736800  18460400  20085900
2016-10-05  113.05  31.59  57.64  21453100  11808600  16726400

In [124]:
stocks.pivot(index = 'Date', columns = 'Symbol', values = 'Close')

Symbol,AAPL,CSCO,MSFT
Date,,,
2016-10-03,112.52,31.50,57.42
2016-10-04,113.00,31.35,57.24
2016-10-05,113.05,31.59,57.64
